In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from tqdm.notebook import tqdm
import copy

device = torch.device(
    'cuda:0' if torch.cuda.is_available() else 'cpu'
)

# Helper functions

In [3]:
def haverside_dist(X1, X2):
    """
    Compute haverside (great circle) distance of locations in X1 and X2, use 
    earth radius R = 6,371 km.
    
    Args:
    X1: lat and lon, shape (m1, 2)
    X2: lat and lon, shape (m2, 2)
    
    Returns:
    ouput: distance matrix, shape (m1, m2)
    """
    R = 6371.
    X1 = X1 * np.pi/180.
    X2 = X2 * np.pi/180.
    
    A = np.cos(X1[:,[1]]) @ np.cos(X2[:,[1]]).T
    A = A * np.cos(X1[:,[0]] - X2[:,[0]].T)
    A += np.sin(X1[:,[1]]) @ np.sin(X2[:,[1]]).T
    A = np.where(A > 1., 1., A)  # for stability    
    
    return R * np.arccos(A)


def mape_loss(pred, target, reduction='mean'):
    """
    input, output: tensor of same shape
    """
    target = torch.where(
        target == 0, 
        torch.tensor(1e-6, device=device), 
        target
    )
    diff = (pred - target) / target
    if reduction == 'mean':
        mape = diff.abs().mean()
    elif reduction == 'sum':
        mape = diff.abs().sum()
    return mape


# Define PM2.5 dataset and dataloader

In [4]:
def get_params(
    loc_df, spatial_res, 
    temporal_res, num_neighbors
):    
    """
    Get parameters of the model.
    
    h, w: height, width of the network
    """
    x0, y0 = 35., 139.8             
    x_max = loc_df.iloc[:, 0].max()  
    y_max = loc_df.iloc[:, 1].max()                           
    h = int((y_max - y0) // spatial_res + 2)
    w = int((x_max - x0) // spatial_res + 2)
    
    return (spatial_res, temporal_res, 
            num_neighbors, h, w)


def create_network(params):   # verified
    """
    Construct a rectangular virtual station network.
    
    net_loc: lat & lon, shape (num_stations, 2) 
    """
    x0, y0 = 35., 139.8     
    spatial_res, _, _, h, w = params
    lat = np.arange(x0, x0 + spatial_res * (w - 0.9), spatial_res)
    lon = np.arange(y0, y0 + spatial_res * (h - 0.9), spatial_res)
    net_loc = np.vstack(
        [np.tile(lat, len(lon)), np.repeat(lon, len(lat))]).transpose()
    return net_loc


def interpolate_pm(
    unknown_loc, known_loc, known_pm, params, reshape=True
):
    """
    Interpolate the PM2.5 values of unknown locations, using k nearest known stations. 
    
    known_pm: array (batch, num_known)
    unkown_pm: array (batch, 1, H, W) if reshape=True, else shape (batch, num_unkown)
    """
    _, _, num_neighbors, h, w = params
    distance = haverside_dist(unknown_loc, known_loc)
    distance = np.where(distance < 1e-6, 1e-6, distance)     
    bound = np.partition(
        distance, num_neighbors - 1, axis=1
    )[:, [num_neighbors - 1]]
    neighbor_mask = np.where(distance <= bound, 1., np.nan)
        
    neighbor_dist = distance * neighbor_mask
    R = 1 / neighbor_dist
    weight = R / np.nansum(R, axis=1, keepdims=True)
    weight = np.nan_to_num(weight, nan=0.)
    
    unknown_pm = known_pm @ weight.T
    if reshape:
        unknown_pm = unknown_pm.reshape([-1, 1, h, w])
    return unknown_pm


def split_dataset(filepath):
    """
    Implement a 60:20:20 contiguous split.
    """
    pm_df = pd.read_csv(
        filepath, header=None, skiprows=1)
    length = len(pm_df)
    train_df = pm_df.loc[: int(0.6 * length)]
    valid_df = pm_df.loc[int(0.6 * length): int(0.8 * length)]
    test_df = pm_df.loc[int(0.8 * length):]
    return train_df, valid_df, test_df


class PMDataset(Dataset):
    def __init__(
        self, pm_df, loc_df, target_idx, test_idx, 
        network_loc, params, training=False
    ):
        """
        pm_df, loc_df: dataframe
        target_idx: integer, test_idx: list
        training: False if test dataset
        """
        self.training = training
        self.window = params[1]
        train_pm, train_loc = self.get_station_data(pm_df, loc_df, train_idx)
        self.input_data = torch.tensor(
            interpolate_pm(network_loc, train_loc, train_pm, params), 
            dtype=torch.float32
        )
        if not self.training:
            test_pm, _ = self.get_station_data(pm_df, loc_df, test_idx)
            self.target_data = torch.tensor(test_pm, dtype=torch.float32)
        else:
            target_pm, target_loc = self.get_station_data(
                pm_df, loc_df, train_idx + [target_idx])
            self.target_data = torch.tensor(
                interpolate_pm(network_loc, target_loc, target_pm, params),
                dtype=torch.float32
            )
    
    @staticmethod
    def get_station_data(pm_df, loc_df, station_idx):
        """
        Retrieve pm2.5 data and locations of given stations.
        """
        station_pm = pm_df.iloc[:, station_idx].to_numpy()
        station_loc = loc_df.iloc[station_idx].to_numpy()
        return station_pm, station_loc
        
    def __len__(self):
        return len(self.input_data) - self.window + 1
    
    def __getitem__(self, idx):
        """
        input shape: (seq_len, 1, H, W)
        target shape: (1, H, W) if testing, else 
        (num_test,)
        """
        input = self.input_data[idx: idx + self.window]
        target = self.target_data[idx + self.window - 1]
        return input, target


# Define layers & model

In [5]:
class TimeDistributed(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.layer = layer

    def forward(self, input):
        """
        input: shape (batch, time, *)
        output: shape (batch, time, *)
        """
        output = []
        for t in range(input.shape[1]):
            output_t = self.layer(input[:, t])
            output_t = output_t.unsqueeze(1)
            output.append(output_t)
        output = torch.cat(output, axis=1)
        return output

        
class tCNN_GRU_Model(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.h, self.w = params[-2:]
        self.conv2d = TimeDistributed(nn.Conv2d(1, 8, 3))
        self.selu1 = TimeDistributed(nn.SELU())
        self.flatten = TimeDistributed(nn.Flatten())
        self.gru = nn.GRU(
            8*(self.h - 2)*(self.w - 2), 100, 2, 
            dropout=0.1
        )
        self.linear = nn.Linear(100, self.h*self.w)
        self.selu2 = nn.SELU()
    
    def forward(self, input):
        """
        input: (batch, seq_len, 1, H, W)
        output: (batch, 1, H, W)
        """
        conv_out = self.conv2d(input)
        conv_out = self.selu1(conv_out)
        conv_out = self.flatten(conv_out)
        conv_out = torch.swapaxes(conv_out, 0, 1)
        gru_out, _ = self.gru(conv_out)
        gru_out = gru_out[-1]
        lin_out = self.linear(gru_out)
        lin_out = self.selu2(lin_out)
        output = lin_out.reshape(-1, 1, self.h, self.w)
        return output


class Predictor(nn.Module):
    def __init__(
        self, loc_df, test_idx, 
        net_loc, params
    ):
        self.test_loc = loc_df.iloc[test_idx].to_numpy()
        self.net_loc = net_loc
        self.params = params

    def __call__(self, net_pm):
        """
        Interpolate test_pm from predicted net_pm.

        net_pm: tensor (batch, 1, H, W)
        """
        net_pm = net_pm.cpu().numpy().reshape(net_pm.shape[0], -1)
        pred = interpolate_pm(
            self.test_loc, self.net_loc, 
            net_pm, self.params, reshape=False
        )
        return torch.from_numpy(pred).to(device)


# Define training/eval loop

In [6]:
def train_loop(model, dataloader, loss_fn, optimizer):
    model.train()
    total_loss = []
    for input, target in dataloader:
        # forward pass
        input = input.to(device)
        target = target.to(device)
        pred = model(input)
        loss = loss_fn(pred, target)
        total_loss.append(loss)
    
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # print loss
    total_loss = torch.tensor(
        total_loss).mean().sqrt().item()
    print(f'train loss: {total_loss:>.4f}')
    return total_loss


def eval_loop(model, dataloader, predictor):
    mse = mae = mape = 0.
    with torch.no_grad():
        test_model = copy.deepcopy(model)
        test_model.eval()
        for input, target in dataloader:
            # forward pass
            input = input.to(device)
            target = target.to(device)
            net_pm = test_model(input)
            pred = predictor(net_pm)
            
            # agregate metrics      
            mse += F.mse_loss(pred, target, reduction='sum')
            mae += F.l1_loss(pred, target, reduction='sum')
            mape += mape_loss(pred, target, reduction='sum')

    num_entries = len(dataloader.dataset) * len(predictor.test_loc)
    rmse = torch.sqrt(mse / num_entries).item()
    mae = (mae / num_entries).item()
    mape = (mape / num_entries).item()
    
    print(f'val loss  : {rmse:>.4f} | {mae:>.4f} | {mape:>.4f}')
    return rmse, mae, mape


# Training time

In [7]:
data_path = ("/content/drive/MyDrive/pm2.5/"
            "data/test_data/(long's)pm2.5.csv")
loc_path = ("/content/drive/MyDrive/pm2.5/"
           "data/test_data/(long's)locations.csv")
test_mode = True
if test_mode:
    data_path = "../data/test_data/long_pm2.5.csv"
    loc_path = "../data/test_data/long_locations.csv"

# train/target/test indices
target_idx = 14                
test_idx = [0,1,4,11,15,24,25,27,32,33,37,39]   
train_idx = list(
    set(range(40)) - set(test_idx) - set([target_idx]))

# hyper-params settings
spatial_res = 0.05
temporal_res = 5
num_neighbors = 10
batch_size = 32
learning_rate = 1e-3

# create net_loc, params
loc_df = pd.read_csv(loc_path, header=None, skiprows=1)
params = get_params(loc_df, spatial_res, temporal_res, num_neighbors)
net_loc = create_network(params)

# create dataset & dataloaders
train_df, valid_df, test_df = split_dataset(data_path)
if test_mode:
    train_df = train_df[:100]
    valid_df = valid_df[:100]
    test_df = test_df[:100]

train_dataset = PMDataset(
    train_df, loc_df, target_idx, test_idx, 
    net_loc, params, training=True
)
valid_dataset = PMDataset(
    valid_df, loc_df, target_idx, test_idx, 
    net_loc, params, training=False
)
test_dataset = PMDataset(
    test_df, loc_df, target_idx, test_idx, 
    net_loc, params, training=False
)

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)

# create training model
model = tCNN_GRU_Model(params)
model = model.to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
predictor = Predictor(loc_df, test_idx, net_loc, params)
train_loss = []

In [8]:
num_epochs = 100
for batch in tqdm(range(num_epochs)):
    batch_loss = train_loop(
        model, train_dataloader, loss_fn, optimizer)
    train_loss.append(batch_loss)
    if epoch % 10 == 9:
        eval_loop(model, valid_dataloader, predictor)
